# Extract from TMDB
* There is no financial information included in the [IMDB data](https://github.com/adeviney/how-to-make-a-successful-movie/blob/main/Data%20Import%20and%20Preprocessing.ipynb) (e.g. budget or revenue). 
- This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before I will be able to determine which movies are successful.
* The Movie Database (TMDB) is great source of financial data (https://www.themoviedb.org/) and offers a free API for programmatic access to their data!
* My stakeholder wants me to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

## Specifications
* My stakeholder would like me to extract and save the results for movies that meet all of the criteria established in the [first part of the project](https://github.com/adeviney/how-to-make-a-successful-movie/blob/main/Data%20Import%20and%20Preprocessing.ipynb). 

* As a proof-of-concept, they requested I perform a test extraction of movies that started in 2000 or 2001

* Each year should be saved as a separate .csv.gz file

In [1]:
# Imports
import numpy as np
import pandas as pd
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
# designate a folder to save information
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'akas.csv',
 'akas.csv.gz',
 'basics.csv',
 'basics.csv.gz',
 'final.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2006.csv.gz',
 'final_tmdb_data_2007.csv.gz',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2020.csv.gz',
 'final_tmdb_data_2021.csv.gz',
 'ratings.csv',
 'ratings.csv.gz',
 'tmdb_2000_2001_results_combined.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003

In [3]:
# define the years to retrieve
YEARS_TO_GET = [2000, 2001]#list(range(2002, 2022))

In [4]:
with open('/Users/Alexis/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [5]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

## Define custom functions

In [6]:
def get_movie_with_rating(movie_id):
    """
    Using tmdbsimple wrapper Python package for TMDB API, most of the data for 
    a movie can be extracted by running the info() method on a movie object.
    However, the certification MPAA rating is not
    The following function will add the MPAA rating of to the dictionary results
    from movie.info() - only if the movie is the US

    Parameters
        ----------
        movie_id : string
             a tconst, an alphanumeric unique identifier of the title, from IMDB

    Returns
        -------
        dict
            the info dict returned from tmdbsimple filtered to only contain 
            revenue, budget, MPAA rating, and imdb_id

    Examples
        --------
        >>> get_movie_with_rating("tt0848228")
        {'adult': False,
        'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
        'belongs_to_collection': {'id': 86311,
        'name': 'The Avengers Collection',
        ...
        'title': 'The Avengers',
        'video': False,
        'vote_average': 7.7,
        'vote_count': 26757,
        'certification': 'PG-13'
        }   
    """
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1'] =='US':
            ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
    return info

In [7]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## Extraction

In [8]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/basics.csv')

for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_request_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    #Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie ids to pandas Series
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    #Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            write_json({'budget': 0, 'revenue': 0, 'imdb_id': movie_id,
                        'certification': 'None'}, JSON_FILE)
            continue
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False) 

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/209 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/242 [00:00<?, ?it/s]